In [1]:
import tensorflow as tf
import os

os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'

MNISTデータセット（手書き文字）をロード。サンプルを整数から浮動小数点数に変換。

In [2]:
tf.keras.backend.set_floatx('float64')

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

層を積み重ねて`tf.keras.Sequential`モデルを構築。訓練のための**オプティマイザ**と**損失関数**を選択。

ReLU
$$
f(x) = \left\{
\begin{array}{ll}
0 & x \leq 0 \\
x & x > 0
\end{array}\right.
$$

$$
f'(x) = \left\{
\begin{array}{ll}
0 & x \leq 0 \\
1 & x > 0
\end{array}\right.
$$

- 平にする層（`Flatten()`）：入力値を1次元ベクトルにする。
- 普通の層（`Dense(units)`）：$\vec{x_{out}} = f(W \vec{x_{in}} + \vec{b})$を計算する。$f(\cdot)$は活性化関数、$W$は重み行列、$\vec{x_{in}}$は入力ベクトル、$\vec{b}$はバイアスベクトルを表す。`units`は出力ベクトルの値の数（ノード数）を表す。
- ドロップアウト層（`Dropout(rate)`）：出力を与えられた確率で0にする。一部のデータが欠損していても正しく認識ができるようになる。これにより、画像の一部の局所特徴が過剰に評価されてしまうのを防ぎ、モデルのロバストさを向上させることができます。

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

`model`はそれぞれの標本についてクラスごとに**ロジット**や**対数オッズ比**と呼ばれるスコアを算出する。

- オッズ：ある事象の起こる確率 $p$ と起こらない確率 $1 − p$ との比 $p / (1 − p)$ のこと。（成功回数）／（失敗回数）
- ロジット：オッズの対数値

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.20917586, -0.09210771,  0.10928842, -0.08397313, -0.55633047,
        -0.39591155, -0.15622961, -0.18255182,  0.2419528 , -0.19919381]])

`tf.nn.softmax`関数はクラスごとにこれらのロジットを確率に変換する。

softmax function ($i = 1, 2, \ldots, n$)
$$
y_i = \frac{e^{x_i}}{\sum_{k=1}^{n} e^{x_k}}
$$

$$
\frac{\partial y_i}{\partial x_j} = \left\{
\begin{array}{ll}
y_i(1-y_j) & (i = j) \\
-y_j y_i & (i \neq j)
\end{array}
\right.
$$

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.13387419, 0.09904922, 0.12114801, 0.09985823, 0.06226467,
        0.07309888, 0.09289734, 0.09048398, 0.13833488, 0.0889906 ]])

損失関数（出力値と正解値との差（距離））`losses.SparseCategoricalCrossentropy`：

ラベルの種類が二つ以上ある場合に使う交差エントロピー関数（教師ラベルは整数値とする）。推定値は各ラベルに当てはまりうる確率を示したベクトル。
$$
H(p, q) = -\sum_{x} p(x) log(q(x))
$$
$p$は真の確率分布、$q$は推定した確率分布。`y_true`は標本数分のラベルのリスト、`y_pred`は(標本数)$\times$(ラベルの種類数)分の確率分布行列。それぞれの標本についてクラスごとに損失のスカラーを出力。

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

この損失は、クラスが正しい確率の対数をとって符号を反転させたもの。この値はモデルがこのクラスが正しいと確信している場合 0 。

この訓練されていないモデルはランダムに近い確率 (それぞれのクラスについて 1/10) を出力、最初の損失は `-tf.log(1/10) ~= 2.3` に近い値になるはず。

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.6159422397613525

- `optimizer`：最適化手法（[tf.keras.optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)）
- `loss`：損失関数（[tf.keras.losses](https://www.tensorflow.org/api_docs/python/tf/keras/losses)）
- `metrics`：学習、テストの時に評価、出力される関数（[tf.keras.metrics](https://www.tensorflow.org/api_docs/python/tf/keras/metrics)）

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

`Model.fit` メソッドは損失を最小化するようにモデルのパラメータを調整する。

In [10]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 12s 199us/sample - loss: 0.2896 - accuracy: 0.9158
Epoch 2/5
60000/60000 [==============================] - 13s 216us/sample - loss: 0.1380 - accuracy: 0.9593
Epoch 3/5
60000/60000 [==============================] - 14s 239us/sample - loss: 0.1042 - accuracy: 0.9690
Epoch 4/5
60000/60000 [==============================] - 15s 253us/sample - loss: 0.0860 - accuracy: 0.9747
Epoch 5/5
60000/60000 [==============================] - 16s 265us/sample - loss: 0.0734 - accuracy: 0.9769


`Model.evaluate` メソッドはモデルの性能を検査する。

これには通常 "検証用データセット" または "テストデータセット" を用いる。

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

10000/1 - 1s - loss: 0.0361 - accuracy: 0.9781


[0.06953047238477622, 0.9781]

この画像識別機はこのデータセットに対して約98%の正解率まで学習できる。

モデルが確率を返すようにしたい場合、モデルをラップしてソフトマックス関数を適用

In [12]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: id=30042, shape=(5, 10), dtype=float64, numpy=
array([[6.71886480e-07, 7.06840195e-07, 7.35471243e-06, 4.55328194e-04,
        1.20977322e-10, 2.16804095e-06, 2.18894875e-12, 9.99513350e-01,
        1.65361123e-05, 3.88428014e-06],
       [5.35206659e-08, 7.46668039e-05, 9.99813001e-01, 1.11330610e-04,
        2.77558231e-15, 5.21970899e-07, 1.21283843e-07, 2.24176201e-12,
        3.05130063e-07, 7.46791547e-12],
       [4.46959457e-08, 9.98602108e-01, 2.58507705e-04, 3.35393254e-06,
        4.56407565e-05, 6.47323332e-06, 6.64008519e-06, 9.66182402e-04,
        1.10436389e-04, 6.12367030e-07],
       [9.99785974e-01, 6.01260411e-09, 9.30764397e-05, 2.72538218e-07,
        6.74453341e-07, 4.07103096e-06, 1.85070835e-05, 9.29116484e-05,
        1.76070800e-07, 4.33085925e-06],
       [5.19838257e-06, 1.91286441e-09, 3.66569739e-06, 1.99544060e-07,
        9.97828008e-01, 6.47876991e-08, 1.31313702e-05, 1.00440136e-05,
        4.85857843e-06, 2.13482761e-03]])>